In [60]:
%matplotlib inline
import pandas
from sklearn.model_selection import train_test_split
import numpy as np
import pylab as pl

In [61]:
song_df = pandas.read_csv('./song_data.csv')

In [62]:
song_df.head()

,user_id,song_id,listen_count,title,artist,song
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Jack Johnson,The Cove - Jack Johnson
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Paco De Lucia,Entre Dos Aguas - Paco De Lucia
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Kanye West,Stronger - Kanye West
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,Jack Johnson,Constellations - Jack Johnson
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODACBL12A8C13C273,1,Learn To Fly,Foo Fighters,Learn To Fly - Foo Fighters


In [63]:
len(song_df)

1116609

In [64]:
song_df = song_df.head(10000)

In [65]:
# Тут пісні сортуються за кількістю прослуховувань з використанням агрегації
song_grouped = song_df.groupby(['song']).agg({'listen_count': 'count'}).reset_index()
grouped_sum  = song_grouped['listen_count'].sum()

song_grouped['percentage']  = song_grouped['listen_count'].div(grouped_sum)*100
song_grouped.sort_values( by='percentage', ascending=False).head(10)

,song,listen_count,percentage
3699,Sehr kosmisch - Harmonia,50,0.50
1067,Dog Days Are Over (Radio Edit) - Florence + Th...,48,0.48
4690,Undo - Björk,46,0.46
5117,You're The One - Dwight Yoakam,43,0.43
3691,Secrets - OneRepublic,42,0.42
3511,Revelry - Kings Of Leon,38,0.38
1889,Horn Concerto No. 4 in E flat K495: II. Romanc...,36,0.36
1407,Fireflies - Charttraxx Karaoke,33,0.33
4412,The Scientist - Coldplay,28,0.28
1829,Hey_ Soul Sister - Train,25,0.25


In [66]:
song_df['song'].describe()

count                        10000
unique                        5155
top       Sehr kosmisch - Harmonia
freq                            50
Name: song, dtype: object

In [67]:
# Кількість записів, а також кількість користувачів
users = song_df['user_id'].unique()

In [68]:
len(song_df)

10000

In [69]:
len(users)

595

### Створення рекомендаційної системи для пісень – simple popularity-based recommender

In [70]:
train_data, test_data = train_test_split(song_df, test_size = 0.20, random_state=0)
train_data.head(5)

,user_id,song_id,listen_count,title,artist,song
7389,354cfdb566f543bb5b810a4d8959d974a30797fd,SOPLUOT12A6D4F7AC3,1,Intergalactic,Beastie Boys,Intergalactic - Beastie Boys
9275,0ec9cc33028dff6209aa49bf645ef64bdcbe00fc,SOHAZRY12A8C13BC47,12,Southside,Common / Kanye West,Southside - Common / Kanye West
2995,8fce200f3912e9608e3b1463cdb9c3529aab5c08,SOIBPQQ12A6310F0F4,1,I Bloom Blaum,Coldplay,I Bloom Blaum - Coldplay
5316,7ef2a3b074b34984f3f677bddde0f1813486cc10,SOUJWJW12A63110848,1,Sound Check (Gravity),Gorillaz,Sound Check (Gravity) - Gorillaz
356,2b6c2f33bc0e887ea7c4411f58106805a1923280,SONYEOJ12A8C142E86,6,Breed,Nirvana,Breed - Nirvana


In [71]:
#Class for Popularity based Recommender System model
class popularity_recommender_py():
    def __init__(self):
        self.train_data = None
        self.user_id = None
        self.item_id = None
        self.popularity_recommendations = None

    #Create the popularity based recommender system model
    def create(self, train_data, user_id, item_id):
        self.train_data = train_data
        self.user_id = user_id
        self.item_id = item_id

        #Get a count of user_ids for each unique song as recommendation score
        train_data_grouped = train_data.groupby([self.item_id]).agg({self.user_id: 'count'}).reset_index()
        train_data_grouped.rename(columns = {'user_id': 'score'},inplace=True)

        #Sort the songs based upon recommendation score
        train_data_sort = train_data_grouped.sort_values(['score', self.item_id], ascending = [0,1])

        #Generate a recommendation rank based upon score
        train_data_sort['Rank'] = train_data_sort['score'].rank(ascending=0, method='first')

        #Get the top 10 recommendations
        self.popularity_recommendations = train_data_sort.head(10)

    #Use the popularity based recommender system model to
    #make recommendations
    def recommend(self, user_id):
        user_recommendations = self.popularity_recommendations

        #Add user_id column for which the recommendations are being generated
        user_recommendations['user_id'] = user_id

        #Bring user_id column to the front
        cols = user_recommendations.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        user_recommendations = user_recommendations[cols]

        return user_recommendations

In [72]:
popularity_m = popularity_recommender_py()
popularity_m.create(train_data, 'user_id', 'song')

In [73]:
popularity_m.recommend(song_df['user_id'][0])

,user_id,song,score,Rank
3228,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Sehr kosmisch - Harmonia,44,1.0
929,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Dog Days Are Over (Radio Edit) - Florence + Th...,40,2.0
4097,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Undo - Björk,35,3.0
3222,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Secrets - OneRepublic,34,4.0
4464,b80344d063b5ccb3212f76538f3d9e43d87dca9e,You're The One - Dwight Yoakam,33,5.0
1231,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Fireflies - Charttraxx Karaoke,29,6.0
3072,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Revelry - Kings Of Leon,29,7.0
1654,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Horn Concerto No. 4 in E flat K495: II. Romanc...,28,8.0
3849,b80344d063b5ccb3212f76538f3d9e43d87dca9e,The Scientist - Coldplay,23,9.0
1002,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Drop The World - Lil Wayne / Eminem,22,10.0


In [74]:
popularity_m.recommend(song_df['user_id'][1])

,user_id,song,score,Rank
3228,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Sehr kosmisch - Harmonia,44,1.0
929,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Dog Days Are Over (Radio Edit) - Florence + Th...,40,2.0
4097,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Undo - Björk,35,3.0
3222,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Secrets - OneRepublic,34,4.0
4464,b80344d063b5ccb3212f76538f3d9e43d87dca9e,You're The One - Dwight Yoakam,33,5.0
1231,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Fireflies - Charttraxx Karaoke,29,6.0
3072,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Revelry - Kings Of Leon,29,7.0
1654,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Horn Concerto No. 4 in E flat K495: II. Romanc...,28,8.0
3849,b80344d063b5ccb3212f76538f3d9e43d87dca9e,The Scientist - Coldplay,23,9.0
1002,b80344d063b5ccb3212f76538f3d9e43d87dca9e,Drop The World - Lil Wayne / Eminem,22,10.0


### Рекомендаційна система з персоналізацією, що заснована на схожості

In [75]:
class item_similarity_recommender_py():
    def __init__(self):
        self.train_data = None
        self.user_id = None
        self.item_id = None
        self.cooccurence_matrix = None
        self.songs_dict = None
        self.rev_songs_dict = None
        self.item_similarity_recommendations = None

    #Get unique items (songs) corresponding to a given user
    def get_user_items(self, user):
        user_data = self.train_data[self.train_data[self.user_id] == user]
        user_items = list(user_data[self.item_id].unique())

        return user_items

    #Get unique users for a given item (song)
    def get_item_users(self, item):
        item_data = self.train_data[self.train_data[self.item_id] == item]
        item_users = set(item_data[self.user_id].unique())

        return item_users

    #Get unique items (songs) in the training data
    def get_all_items_train_data(self):
        all_items = list(self.train_data[self.item_id].unique())

        return all_items

    #Construct cooccurence matrix
    def construct_cooccurence_matrix(self, user_songs, all_songs):

        ####################################
        #Get users for all songs in user_songs.
        ####################################
        user_songs_users = []
        for i in range(0, len(user_songs)):
            user_songs_users.append(self.get_item_users(user_songs[i]))

        ###############################################
        #Initialize the item cooccurence matrix of size
        #len(user_songs) X len(songs)
        ###############################################
        cooccurence_matrix = np.matrix(np.zeros(shape=(len(user_songs), len(all_songs))), float)

        #############################################################
        #Calculate similarity between user songs and all unique songs
        #in the training data
        #############################################################
        for i in range(0,len(all_songs)):
            #Calculate unique listeners (users) of song (item) i
            songs_i_data = self.train_data[self.train_data[self.item_id] == all_songs[i]]
            users_i = set(songs_i_data[self.user_id].unique())

            for j in range(0,len(user_songs)):

                #Get unique listeners (users) of song (item) j
                users_j = user_songs_users[j]

                #Calculate intersection of listeners of songs i and j
                users_intersection = users_i.intersection(users_j)

                #Calculate cooccurence_matrix[i,j] as Jaccard Index
                if len(users_intersection) != 0:
                    #Calculate union of listeners of songs i and j
                    users_union = users_i.union(users_j)

                    cooccurence_matrix[j,i] = float(len(users_intersection))/float(len(users_union))
                else:
                    cooccurence_matrix[j,i] = 0


        return cooccurence_matrix


    #Use the cooccurence matrix to make top recommendations
    def generate_top_recommendations(self, user, cooccurence_matrix, all_songs, user_songs):
        print("Non zero values in cooccurence_matrix :%d" % np.count_nonzero(cooccurence_matrix))

        #Calculate a weighted average of the scores in cooccurence matrix for all user songs.
        user_sim_scores = cooccurence_matrix.sum(axis=0)/float(cooccurence_matrix.shape[0])
        user_sim_scores = np.array(user_sim_scores)[0].tolist()

        #Sort the indices of user_sim_scores based upon their value
        #Also maintain the corresponding score
        sort_index = sorted(((e,i) for i,e in enumerate(list(user_sim_scores))), reverse=True)

        #Create a dataframe from the following
        columns = ['user_id', 'song', 'score', 'rank']
        #index = np.arange(1) # array of numbers for the number of samples
        df = pandas.DataFrame(columns=columns)

        #Fill the dataframe with top 10 item based recommendations
        rank = 1
        for i in range(0,len(sort_index)):
            if ~np.isnan(sort_index[i][0]) and all_songs[sort_index[i][1]] not in user_songs and rank <= 10:
                df.loc[len(df)]=[user,all_songs[sort_index[i][1]],sort_index[i][0],rank]
                rank = rank+1

        #Handle the case where there are no recommendations
        if df.shape[0] == 0:
            print("The current user has no songs for training the item similarity based recommendation model.")
            return -1
        else:
            return df

    #Create the item similarity based recommender system model
    def create(self, train_data, user_id, item_id):
        self.train_data = train_data
        self.user_id = user_id
        self.item_id = item_id

    #Use the item similarity based recommender system model to
    #make recommendations
    def recommend(self, user):

        ########################################
        #A. Get all unique songs for this user
        ########################################
        user_songs = self.get_user_items(user)

        print("No. of unique songs for the user: %d" % len(user_songs))

        ######################################################
        #B. Get all unique items (songs) in the training data
        ######################################################
        all_songs = self.get_all_items_train_data()

        print("No. of unique songs in the training set: %d" % len(all_songs))

        ###############################################
        #C. Construct item cooccurence matrix of size
        #len(user_songs) X len(songs)
        ###############################################
        cooccurence_matrix = self.construct_cooccurence_matrix(user_songs, all_songs)

        #######################################################
        #D. Use the cooccurence matrix to make recommendations
        #######################################################
        df_recommendations = self.generate_top_recommendations(user, cooccurence_matrix, all_songs, user_songs)

        return df_recommendations

    #Get similar items to given items
    def get_similar_items(self, item_list):

        user_songs = item_list

        ######################################################
        #B. Get all unique items (songs) in the training data
        ######################################################
        all_songs = self.get_all_items_train_data()

        print("No. of unique songs in the training set: %d" % len(all_songs))

        ###############################################
        #C. Construct item cooccurence matrix of size
        #len(user_songs) X len(songs)
        ###############################################
        cooccurence_matrix = self.construct_cooccurence_matrix(user_songs, all_songs)

        #######################################################
        #D. Use the cooccurence matrix to make recommendations
        #######################################################
        user = ""
        df_recommendations = self.generate_top_recommendations(user, cooccurence_matrix, all_songs, user_songs)

        return df_recommendations

In [76]:
personalized_model = item_similarity_recommender_py()
personalized_model.create( train_data, 'user_id', 'song' )

In [77]:
cur_user_id = users[5]
user_items = personalized_model.get_user_items( cur_user_id )
personalized_model.recommend( cur_user_id )

No. of unique songs for the user: 13
No. of unique songs in the training set: 4496
Non zero values in cooccurence_matrix :959


,user_id,song,score,rank
0,4bd88bfb25263a75bbdd467e74018f4ae570e5df,You Found Me (Album Version) - The Fray,0.058974,1
1,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Hellbound - J-Black & Masta Ace,0.058974,2
2,4bd88bfb25263a75bbdd467e74018f4ae570e5df,As The World Turns - Eminem,0.053780,3
3,4bd88bfb25263a75bbdd467e74018f4ae570e5df,I'm Back - Eminem,0.052015,4
4,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Armed And Ready (2009 Digital Remaster) - The ...,0.044322,5
5,4bd88bfb25263a75bbdd467e74018f4ae570e5df,I'm The One Who Understands (Edit Version) - War,0.044322,6
6,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Falling - Iration,0.044322,7
7,4bd88bfb25263a75bbdd467e74018f4ae570e5df,Superman - Eminem / Dina Rae,0.039777,8
8,4bd88bfb25263a75bbdd467e74018f4ae570e5df,The Seed (2.0) - The Roots / Cody Chestnutt,0.038586,9
9,4bd88bfb25263a75bbdd467e74018f4ae570e5df,The Way You Move - OutKast,0.038586,10


In [78]:
personalized_model.get_similar_items(['Chan Chan (Live) - Buena Vista Social Club'])

No. of unique songs in the training set: 4496
Non zero values in cooccurence_matrix :22


,user_id,song,score,rank
0,,Always Like This - Bombay Bicycle Club,0.500000,1
1,,Kong - Bonobo,0.500000,2
2,,I'll Try Anything Once - The Strokes,0.500000,3
3,,Cotton Fields (The Cotton Song) (Digitally Rem...,0.500000,4
4,,You Only Live Once - The Strokes,0.500000,5
5,,The Sound of Settling (Album Version) - Death ...,0.500000,6
6,,Novocaine For The Soul - Eels,0.333333,7
7,,Every Lasting Light - The Black Keys,0.333333,8
8,,This Charming Man - The Smiths,0.333333,9
9,,Holland_ 1945 - Neutral Milk Hotel,0.333333,10
